In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 
        # 3x3 square convolution kerel
        
        #nn.Conv2d(in_channels, out_channels, kernel_size, 
        #stride=1, padding =0, dilation=1, groups=1, bias=True)
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)    # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square you can only specify a single num
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) # flatten the matrix before FC layer
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net() # instantiate the network class
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
# The learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weights
        

10
torch.Size([6, 1, 3, 3])


In [28]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0429, -0.1791, -0.0302, -0.0152, -0.0738, -0.1068, -0.0428, -0.0156,
          0.0576,  0.0114]], grad_fn=<AddmmBackward>)


In [29]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [35]:
# loss function

output = net(input)
target = torch.randn(10)   # a dummy target for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7829, grad_fn=<MseLossBackward>)


In [36]:
#when calling loss.backward(), the whole graph is differentiated

print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU


In [37]:
# backprop
net.zero_grad()  # zeroes the grad buffers so that they dont accumulate

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0065,  0.0065,  0.0165, -0.0187, -0.0191,  0.0114])


In [40]:
# updating the weights
# weight = weight - learning_rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(learning_rate * f.grad.data)

In [ ]:
# optimizer

import torch.optim as optim

# create your optimizer, SGD, Adam, RMSProp, etc
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zeroes the grad buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # does the update